# Imports

In [1]:
import sys
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

In [2]:
sys.executable

'/Users/imad/anaconda3/envs/ml-zoo/bin/python'

In [3]:
import sklearn
sklearn.__version__

'0.21.2'

# Data

In [4]:
train_df = pd.read_csv('../data/train.csv')
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
house_style = train_df[['HouseStyle']].copy()
ohe = OneHotEncoder(sparse=False)
house_style_transformed = ohe.fit_transform(house_style)

In [6]:
ohe.get_feature_names()

array(['x0_1.5Fin', 'x0_1.5Unf', 'x0_1Story', 'x0_2.5Fin', 'x0_2.5Unf',
       'x0_2Story', 'x0_SFoyer', 'x0_SLvl'], dtype=object)

In [7]:
pd.Series(house_style.values.flatten()).unique()

array(['2Story', '1Story', '1.5Fin', '1.5Unf', 'SFoyer', 'SLvl', '2.5Unf',
       '2.5Fin'], dtype=object)

In [8]:
set(pd.Series(ohe.get_feature_names()).str.replace('x0_', '')) ==  set(pd.Series(house_style.values.flatten()).unique())

True

In [9]:
y = train_df.pop('SalePrice').values

In [10]:
# Get the kind of each feature
kinds = np.array([dt.kind for dt in train_df.dtypes])
kinds

array(['i', 'i', 'O', 'f', 'i', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
       'O', 'O', 'O', 'O', 'i', 'i', 'i', 'i', 'O', 'O', 'O', 'O', 'O',
       'f', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'i', 'O', 'i', 'i', 'i',
       'O', 'O', 'O', 'O', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i',
       'i', 'O', 'i', 'O', 'i', 'O', 'O', 'f', 'O', 'i', 'i', 'O', 'O',
       'O', 'i', 'i', 'i', 'i', 'i', 'i', 'O', 'O', 'O', 'i', 'i', 'i',
       'O', 'O'], dtype='<U1')

In [11]:
features = train_df.columns.values
is_num = kinds != 'O'
num_feat = features[is_num]
cat_feat = features[~is_num]

In [12]:
num_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('std', StandardScaler())
])
cat_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='constant', fill_value='MISSING')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [13]:
transformer = ColumnTransformer([
    ('num', num_pipe, num_feat),
    ('cat', cat_pipe, cat_feat)
])

In [14]:
ml_pipe = Pipeline([
    ('transform', transformer),
    ('model', Ridge())
])

In [15]:
ml_pipe.fit(train_df, y)

Pipeline(memory=None,
         steps=[('transform',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imp',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                    

In [16]:
ml_pipe.score(train_df, y)

0.9220545988101001

In [17]:
ml_pipe.named_steps['transform'].named_transformers_['cat'].named_steps['ohe'].get_feature_names()

array(['x0_C (all)', 'x0_FV', 'x0_RH', 'x0_RL', 'x0_RM', 'x1_Grvl',
       'x1_Pave', 'x2_Grvl', 'x2_MISSING', 'x2_Pave', 'x3_IR1', 'x3_IR2',
       'x3_IR3', 'x3_Reg', 'x4_Bnk', 'x4_HLS', 'x4_Low', 'x4_Lvl',
       'x5_AllPub', 'x5_NoSeWa', 'x6_Corner', 'x6_CulDSac', 'x6_FR2',
       'x6_FR3', 'x6_Inside', 'x7_Gtl', 'x7_Mod', 'x7_Sev', 'x8_Blmngtn',
       'x8_Blueste', 'x8_BrDale', 'x8_BrkSide', 'x8_ClearCr',
       'x8_CollgCr', 'x8_Crawfor', 'x8_Edwards', 'x8_Gilbert',
       'x8_IDOTRR', 'x8_MeadowV', 'x8_Mitchel', 'x8_NAmes', 'x8_NPkVill',
       'x8_NWAmes', 'x8_NoRidge', 'x8_NridgHt', 'x8_OldTown', 'x8_SWISU',
       'x8_Sawyer', 'x8_SawyerW', 'x8_Somerst', 'x8_StoneBr', 'x8_Timber',
       'x8_Veenker', 'x9_Artery', 'x9_Feedr', 'x9_Norm', 'x9_PosA',
       'x9_PosN', 'x9_RRAe', 'x9_RRAn', 'x9_RRNe', 'x9_RRNn',
       'x10_Artery', 'x10_Feedr', 'x10_Norm', 'x10_PosA', 'x10_PosN',
       'x10_RRAe', 'x10_RRAn', 'x10_RRNn', 'x11_1Fam', 'x11_2fmCon',
       'x11_Duplex', 'x11_Twnh